### **Task 2: Car Counter**

**Objectives:**  
1. Bounding box
2. Total car count

**Media:**   
videos/video.mp4

**Approaches:**  
1. Object Detection + Object Tracking + Object Counting
2. Brackground Substraction + Object Tracking + Object Counting

**References**
1. https://www.v7labs.com/blog/yolo-object-detection
2. https://www.analyticsvidhya.com/blog/2020/08/selecting-the-right-bounding-box-using-non-max-suppression-with-implementation/
3. https://docs.ultralytics.com/reference/solutions/object_counter/#ultralytics.solutions.object_counter.ObjectCounter.count
4. https://towardsdatascience.com/mastering-object-counting-in-videos-3d49a9230bd2/

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

##### **2.1 Object Detection + Object Tracking + Object Counting**

Result: Accurate but slow

In [2]:
# Load YOLO model
model = YOLO(model="yolo11n.pt")  

# Initialize DeepSORT tracker
tracker = DeepSort()

# Open video
cap = cv2.VideoCapture("../videos/video.mp4")

# Define counting line (x-coordinate)
line_position = 320  
counted_ids = set()  
object_count = 0  

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    frame = cv2.resize(frame, (640, 480))
    
    # Step 1: Detect objects using YOLO
    results = model(frame, iou=0.2, conf=0.5)  
    detections = []  

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box
            conf = box.conf[0].item()  # Confidence score
            cls = int(box.cls[0].item())  # Class ID
            detections.append(([x1, y1, x2-x1, y2-y1], conf, cls))

    # Step 2: Track objects using DeepSORT
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        x1, y1, x2, y2 = map(int, track.to_tlbr())  # Bounding box
        track_id = track.track_id  # Unique ID assigned by DeepSORT
        centroid_x = (x1 + x2) // 2  # Object's center X position

        # Step 3: Check if object crosses the line
        if track_id not in counted_ids and centroid_x > line_position:
            counted_ids.add(track_id)  # Mark ID as counted
            object_count += 1  # Increase count

        # Draw bounding box and ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display count
    cv2.putText(frame, f"Count: {object_count}", (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow("Object Counting", frame)
    if cv2.waitKey(1) & 0xFF == 27:  
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 1 car, 132.0ms
Speed: 1.8ms preprocess, 132.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 140.9ms
Speed: 2.3ms preprocess, 140.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 77.9ms
Speed: 1.2ms preprocess, 77.9ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 88.3ms
Speed: 1.3ms preprocess, 88.3ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 96.7ms
Speed: 1.4ms preprocess, 96.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 87.3ms
Speed: 1.3ms preprocess, 87.3ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 88.9ms
Speed: 1.3ms preprocess, 88.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 149.2ms
Speed: 1.3ms preprocess, 149.2ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x64

##### **2.2 Brackground Substraction + Object Tracking + Object Counting (Cannot be used)**

Result: Fast but inaccurate  

Debug:
- Bounding box ok (but too big)
- Sudden pixel change when approaching border due to inappropriate morphological operation (maybe?)
- So causing the creation of > 1 box for the same car

In [ ]:
video_path = '../videos/video.mp4'
cap = cv2.VideoCapture(video_path)

substractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=175, detectShadows=False) # Due to outdoor shooting
tracker = DeepSort()
kernel = np.ones((3, 3), np.uint8)
total_car = 0
x_passing_line = 320
counted_ids = set()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Preprocessing
    frame = cv2.resize(frame, (640, 480))
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Background subtraction
    fgmask = substractor.apply(image=hsv_frame, learningRate=0.2)

    # Morphological operations
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    fgmask = cv2.medianBlur(fgmask, 3)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel, iterations=90)

    # Detect objects using contours
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detections = []

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt) 
        if cv2.contourArea(cnt) > 2000 and h<w:  
            detections.append(([x, y, x + w, y + h], 1.0, None))  # Format for DeepSORT

    # Step 2: Track objects using DeepSORT
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        x1, y1, x2, y2 = map(int, track.to_tlbr())  # Bounding box
        track_id = track.track_id  # Unique ID assigned by DeepSORT
        centroid_x = (x1 + x2) // 2  # Object's center X position

        # Step 3: Check if object crosses the line
        if track_id not in counted_ids and centroid_x > x_passing_line:
            counted_ids.add(track_id)  # Mark ID as counted
            total_car += 1  # Increase count

        # Draw bounding box and ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display count
    cv2.putText(frame, f"Count: {total_car}", (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow("Object Counting", frame)
    cv2.imshow("Foreground Mask", fgmask)
    
    if cv2.waitKey(1) & 0xFF == 27: 
        break

cap.release()
cv2.destroyAllWindows()